In [11]:
from datetime import datetime

In [12]:
datetime.now()

datetime.datetime(2023, 1, 23, 15, 49, 48, 622675)

In [ ]:
from typing import Callable
import numpy as np

# This is our decorator
def submit_to_domain_node(user_func: Callable, user_input: np.ndarray, policy, constraint):
    print("We would try to check that the verify key is correct")
    print("We 

In [1]:
def parent(num):
    
    def first_child():
        return "Emily"
    
    def second_child():
        return "George"
    
    if num == 1:
        return first_child
    else:
        return second_child

In [5]:
def decorator(func):
    def wrapper():
        print("Something happens before the function")
        func()
        print("Something happens after the function")
    return wrapper

In [6]:
def say_whee():
    print("Whee!")

say_whee = decorator(say_whee)

In [7]:
say_whee()

Something happens before the function
Whee!
Something happens after the function


In [8]:
@decorator
def say_boo():
    print("Booo!")

In [10]:
say_boo()

Something happens before the function
Booo!
Something happens after the function
